In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from peft import PeftModel, PeftConfig
import torch
import gc

# Load tokenizer
model_name = "meta-llama/Llama-3.2-3B-Instruct"  # or the base model you trained on
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Load base model in 4-bit if you used quantization
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
config = AutoConfig.from_pretrained(model_name)
# manually set rope_scaling to supported structure:
config.rope_scaling = {"type": "dynamic", "factor": 2.0}
config.use_cache = True

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    config = config,
    torch_dtype=torch.float16
)
adapted_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    config=config,
    torch_dtype=torch.float16
)

# Load your LoRA adapter
adapter_path = "./../Training/final_adapter_with_eval_0"  # or wherever your adapter_model.safetensors is
adapted_model= PeftModel.from_pretrained(adapted_model, adapter_path)


2025-03-26 15:18:42.147556: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
#Need to empty_cache other wise 2 models' results will bleed into each other
# only slice and decode the new tokens 
def generate_summary(input_text, max_new_tokens=150):
    gc.collect()
    torch.cuda.empty_cache()
    prompt = f"""Summarize:\n{input_text} Summary:\n"""

#     prompt = f"""Without commentary, from its original language summarize to English on useful information including sensitive data, below 100 words. If no meaning return <NULL>
# Text:
# {input_text}

    
    inputs = tokenizer(prompt, return_tensors="pt").to(adapted_model.device)
    
    with torch.no_grad():
        outputs = adapted_model.generate(
            **inputs,
            do_sample=True,
            temperature=0.7,
            max_new_tokens=max_new_tokens,
            top_p=0.9
        )
    input_len = inputs["input_ids"].shape[1]
    new_tokens = outputs[0][input_len:]  # exclude prompt
    summary = tokenizer.decode(new_tokens, skip_special_tokens=True)
    return summary

def generate_base_summary(input_text, max_new_tokens=150):
    gc.collect()
    torch.cuda.empty_cache()

    prompt = f"""Summarize:\n{input_text} Summary:\n"""
#     prompt = f"""Without commentary, from its original language summarize to English on useful information including sensitive data, below 100 words. If no meaning return <NULL>
# Text:
# {input_text}
# """
    
    inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)
    
    with torch.no_grad():
        outputs = base_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    input_len = inputs["input_ids"].shape[1]
    new_tokens = outputs[0][input_len:]  # exclude prompt
    summary = tokenizer.decode(new_tokens, skip_special_tokens=True)
    return summary
    

In [3]:
import json
def get_json_from_file(file_path):

    with open(file_path, "r", encoding="utf-8") as file:
        data = file.read().strip()

        # Fix concatenated JSON objects
        objs = data.split("}{")

        parsed_objs = []
        for i, obj in enumerate(objs):
            if not obj.startswith("{"):
                obj = "{" + obj
            if not obj.endswith("}"):
                obj = obj + "}"
            try:
                parsed = json.loads(obj)
                parsed_objs.append(parsed)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {file_path}, object {i}: {e}")
    return parsed_objs

In [4]:

# example_file = "./../Training/../Training/InputNLabel/prompt_sensitive_translated/receipts_2.json"
# file_and_limits = {
#     "./../Training/InputNLabel/filtered_labels/filtered_formatted.json": 30,
#     "./../Training/InputNLabel/filtered_labels/filtered_randoms.json": 30,
#     "./../Training/InputNLabel/filtered_labels/filtered_receipts.json": 30,
#     "./../Training/InputNLabel/filtered_labels/filtered_reports.json": 10,
#     "./../Training/InputNLabel/prompt_sensitive_translated/formatted_2.json": 300,
#     "./../Training/InputNLabel/prompt_sensitive_translated/random_2.json": 300,
#     "./../Training/InputNLabel/prompt_sensitive_translated/receipts_2.json": 300,
#     "./../Training/InputNLabel/prompt_sensitive_translated/reports_2.json": 300,
# }
formatted_chunks = "./../Training/InputNLabel/prompt_sensitive_translated/formatted_2.json"
receipt_chunks = "./../Training/InputNLabel/prompt_sensitive_translated/receipts_2.json"
report_chunks = "./../Training/InputNLabel/prompt_sensitive_translated/reports_2.json"
random_chunks = "./../Training/InputNLabel/prompt_sensitive_translated/random_2.json"

filtered_receipts = "./../Training/InputNLabel/filtered_labels/filtered_receipts.json"
filtered_reports = "./../Training/InputNLabel/filtered_labels/filtered_reports.json"
filtered_randoms = "./../Training/InputNLabel/filtered_labels/filtered_randoms.json"
filtered_formatted = "./../Training/InputNLabel/filtered_labels/filtered_formatted.json"

formatted = get_json_from_file(formatted_chunks)
receipts = get_json_from_file(receipt_chunks)
reports = get_json_from_file(report_chunks)
randoms = get_json_from_file(random_chunks)

f_receipts = get_json_from_file(filtered_receipts)
f_reports = get_json_from_file(filtered_reports)
f_randoms = get_json_from_file(filtered_randoms)
f_formatted = get_json_from_file(filtered_formatted)

print("chunks with formats",len(formatted))
print("chunks of recipts that need ocr", len(receipts))
print("chunks of well written long papers, no ocr",len(reports))
print("chunks of all sorts",len(randoms))

print("chunks with filtered formats",len(f_formatted))
print("chunks with formatted receipts", len(f_receipts))
print("chunks with formatted reports", len(f_reports))
print("chunks with formatted randoms", len(f_randoms))



chunks with formats 300
chunks of recipts that need ocr 300
chunks of well written long papers, no ocr 300
chunks of all sorts 300
chunks with filtered formats 99
chunks with formatted receipts 100
chunks with formatted reports 30
chunks with formatted randoms 100


##### Testing filtered chunks

In [5]:
input = f_receipts[0]
print(input["input"])
print("\nOutput:",input["output"])

MARKET
Pesan as 62-2104
se
etna
saa
10h
czy corn Eueees ” a
Raurommcorse =|
Aur isto.
|| B
ferecth a ne bibs
B
Stl
a
ate:
f
ital
S
Ei iam:
paid
Riles,
ase
1
ip facoet mor
his Er i Amn
rug;
AIT retro a e
Wrveuty ov tee fc 
3B Ghar asia tea
biti vat
canter
fc} tars ave a ele Taos
a's Pras Stig, Vid tke
{Sin ie a scon
Tam 5 ack at ne fons e
HR the tan Pres hoar
(Shave a musi.cofet

--------------------------------------------------

Output: <NULL>


In [9]:
input_text = input["input"]
#print(generate_base_summary(input_text,150))
print("!!!!<---------Next Summary----------->!!!!")
print(generate_summary(input_text, 150))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


!!!!<---------Next Summary----------->!!!!
Summary:
<NULL>


In [8]:
print(f_reports[0]["input"])
print(f_receipts[0]["output"])


---------------------
<NULL>


#### Test Runs of trained adapter

In [5]:
input = receipts[0]
#print(generate_base_summary(input["input"]))
print("!!!!<----------------------->!!!!")
print(generate_summary(input["input"]))

!!!!<----------------------->!!!!


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KFC Nederland, located at KFC 'Jan van GaJen, Restaurant number 222, has a receipt (Chk 5460) for an order made on Oct09'19 at 06:58. The order includes 1 Upt 3 Hot Wings at EUR1.95, 1 Fire Stack Ch at EUR10.65, Pepsi Max - L at EUR0.65, and Fries at EUR0.65. The total payment was EUR12.60, paid by Master Card. The receipt also mentions a 9% VAT and a 2-minute wait. The customer is offered 3 free Hot Wings with their next order over EUR6. The receipt is closed at 18:58. Thank


In [5]:
#trained adapter 0 after 72 examples: there is still original text but a summary is now  more concise
input_text = formatted[0]["input"] + formatted[1]["input"] + formatted[2]["input"]

#input_text = f_receipts[0]["input"]
print("!!!!<----------------------->!!!!")
#print(generate_base_summary(input_text))
print("!!!!<----------------------->!!!!")
summary_0 = generate_summary(formatted[0]["input"])
print("0:",summary_0)
summary_1 = generate_summary(formatted[1]["input"])
print("1:",summary_1)
summary_2 = generate_summary(formatted[2]["input"])
print("2:", summary_2)
print(generate_summary(summary_0 + summary_1 + summary_2))

!!!!<----------------------->!!!!
!!!!<----------------------->!!!!


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


0: The Jack's New England Glam Chowder was shipped on August 11, 2017. The product was sold in a unit of 20, with each unit priced at 9.65. No further information is provided.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1: The shipment took place on 2017-08-28. The products included were Gudbrandsdalsost with a quantity of 20 units at a unit price of 36.0, totaling 720.0, and Outback Lager with a quantity of 15 units at a unit price of 15.0, totaling 225.0. The overall total price was 945.0.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


2: Order ID 10481 was placed by Ricardo Adocicados (Customer ID: RICAR) from Av. Copacabana, 257, de Janeiro, Brazil on 2017-03-20 and shipped by United Package (Shipper ID: 2) on 2017-03-25. The order included 26 units of Maxiak at a unit price of 160. The order was handled by employee Laura Callahan. The shipping address is 02389-890, South America.
The Jack's New England Glam Chowder was shipped on August 11, 2017. The product was sold in a unit of 20, with each unit priced at 9.65. The shipment took place on 2017-08-28. The products included were Gudbrandsdalsost with a quantity of 20 units at a unit price of 36.0, totaling 720.0, and Outback Lager with a quantity of 15 units at a unit price of 15.0, totaling 225.0. The overall total price was 945.0. Order ID 10481 was placed by Ricardo Adocicados (Customer ID: RICAR) from Av. Copacab


In [6]:
print("!!!!<----------------------->!!!!")
print(generate_base_summary(input_text))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


!!!!<----------------------->!!!!
The customer, Ricardo Adocicados, has placed an order for the following products:
- Maxiak (26 units)
- Gudbrandsdalsost (20 units)
- Outback Lager (15 units)
The order was shipped on March 25, 2017, from the United Package shipping company.
The total price of the order is $945.00.
The customer's shipping details are:
- Ship Name: Ricardo Adocicados
- Ship Address: Av. Copacabana, 257
- Ship City: de Janeiro
- Ship Postal Code: 02389-890
- Ship County: Brazil
- Ship Region: South America
The customer's order details are:
- Order Date:


In [7]:
print("!!!!<----------------------->!!!!")
print(generate_summary(input_text))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


!!!!<----------------------->!!!!
The text is a shipping order from Ricardo Adocicados to United Package. The order was placed on 2017-03-20 and shipped on 2017-03-25. The order included two products: Maxiak with a quantity of 26 and unit price of 160.0, and Outback Lager with a quantity of 15 and unit price of 15.0. The total price was 945.0. The shipping details were sent to Ricardo Adocicados at Av. Copacabana, 257, Copacabana, de Janeiro, South America, 02389-890, Brazil. The order was handled by employee Laura Callahan and shipped by United Package.


In [ ]:
#Summary  from one recursive pass of 3 summaries of original text 
"""The Jack's New England Glam Chowder was shipped on August 11, 2017. The product was sold in a unit of 20, with each unit priced at 9.65. The shipment took place on 2017-08-28. The products included were Gudbrandsdalsost with a quantity of 20 units at a unit price of 36.0, totaling 720.0, and Outback Lager with a quantity of 15 units at a unit price of 15.0, totaling 225.0. The overall total price was 945.0. Order ID 10481 was placed by Ricardo Adocicados (Customer ID: RICAR) from Av. Copacab"""

#Summary  of adapted model from 1 pass of 3  chunks of original text 
"""The text is a shipping order from Ricardo Adocicados to United Package. The order was placed on 2017-03-20 and shipped on 2017-03-25. The order included two products: Maxiak with a quantity of 26 and unit price of 160.0, and Outback Lager with a quantity of 15 and unit price of 15.0. The total price was 945.0. The shipping details were sent to Ricardo Adocicados at Av. Copacabana, 257, Copacabana, de Janeiro, South America, 02389-890, Brazil. The order was handled by employee Laura Callahan and shipped by United Package."""

#Summary of base model from 3 chunks of original text
"""
The customer, Ricardo Adocicados, has placed an order for the following products:
- Maxiak (26 units)
- Gudbrandsdalsost (20 units)
- Outback Lager (15 units)
The order was shipped on March 25, 2017, from the United Package shipping company.
The total price of the order is $945.00.
The customer's shipping details are:
- Ship Name: Ricardo Adocicados
- Ship Address: Av. Copacabana, 257
- Ship City: de Janeiro
- Ship Postal Code: 02389-890
- Ship County: Brazil
- Ship Region: South America
The customer's order details are:
- Order Date:
"""
